In [6]:
import numpy as np
import pandas as pd

def price_defaultable_zcb(n, r0, u, d, q, a, b, F=100.0, R=0.20):
    """
    V[i,j] = [ (1-h_ij) * E_q[V[i+1,*]] + h_ij * R*F ] / (1 + r_ij),
    with terminal V[n,j] = F.
    """
    # terminal values
    V = [np.full(i+1, F, dtype=np.float64) for i in range(n+1)]  # V[n] is length n+1
    # backward induction
    for i in range(n-1, -1, -1):
        V_next = V[i+1]
        V_i = np.empty(i+1, dtype=np.float64)
        if i == n-1:
            r_lattice = pd.DataFrame(index=range(n+1), columns=range(n+1))
            h_lattice = pd.DataFrame(index=range(n+1), columns=range(n+1))
        for j in range(i+1):
            r_ij = r0 * (u**j) * (d**(i-j))          # per‑step short rate (decimal)
            h_ij = a * (b**(j - 0.5 * i))            # updated hazard function
            h_ij = float(min(max(h_ij, 0.0), 1.0))   # clip to [0,1]
            cont = q * V_next[j+1] + (1.0-q) * V_next[j]
            r_lattice.iloc[j, i] = r_ij
            h_lattice.iloc[j, i] = h_ij
            V_i[j] = ((1.0 - h_ij) * cont + h_ij * R * F) / (1.0 + r_ij)
        V[i] = V_i
    print("Hazard rates")
    print(h_lattice)
    print("Short rates")
    print(r_lattice)
    print("Bond prices")
    print(V_i)
    return V[0][0]

# parameters from the prompt
n = 10
r0, u, d, q = 0.05, 1.1, 0.9, 0.5
a, b = 0.01, 1.01
F, R = 100.0, 0.20

price = price_defaultable_zcb(n, r0, u, d, q, a, b, F, R)
print(round(price, 2))

Hazard rates
      0        1         2         3         4         5         6         7   \
0   0.01  0.00995  0.009901  0.009852  0.009803  0.009754  0.009706  0.009658   
1    NaN  0.01005      0.01   0.00995  0.009901  0.009852  0.009803  0.009754   
2    NaN      NaN    0.0101   0.01005      0.01   0.00995  0.009901  0.009852   
3    NaN      NaN       NaN   0.01015    0.0101   0.01005      0.01   0.00995   
4    NaN      NaN       NaN       NaN  0.010201   0.01015    0.0101   0.01005   
5    NaN      NaN       NaN       NaN       NaN  0.010252  0.010201   0.01015   
6    NaN      NaN       NaN       NaN       NaN       NaN  0.010303  0.010252   
7    NaN      NaN       NaN       NaN       NaN       NaN       NaN  0.010354   
8    NaN      NaN       NaN       NaN       NaN       NaN       NaN       NaN   
9    NaN      NaN       NaN       NaN       NaN       NaN       NaN       NaN   
10   NaN      NaN       NaN       NaN       NaN       NaN       NaN       NaN   

          8   